In [1]:
from sklearn.datasets import make_regression
from keras.models import Sequential
from keras.layers import Dense
import seaborn as sns
import pandas as pd
from math import e,sqrt
from scipy.optimize import least_squares
from scipy.optimize import leastsq

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from pyFTS.models.nonstationary import nsfts
from pyFTS.benchmarks import Measures
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import datetime
import statistics
import math
import os
import sys
import statistics
sys.path.append("/home/hugo/projetos-doutorado/mimo_emb_fts/src/")

from embfts.util.DataSetUtil import DataSetUtil
from embfts.util.StatisticsUtil import StatisticsUtil

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor


In [2]:
data_set_util = DataSetUtil()
statistics_util = StatisticsUtil()

In [3]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

In [4]:
df = pd.read_csv('/home/hugo/projetos-doutorado/mimo_emb_fts/data/air/air_quality_beijing_6_sites.csv', sep=',')
df = df.drop(labels=['No','day','year','month','hour','wd','station'], axis=1)
df = df.drop(labels=['No.1','day.1','year.1','month.1','hour.1','wd.1','station.1'], axis=1)
df = df.drop(labels=['No.2','day.2','year.2','month.2','hour.2','wd.2','station.2'], axis=1)
df = df.drop(labels=['No.3','day.3','year.3','month.3','hour.3','wd.3','station.3'], axis=1)
df = df.drop(labels=['No.4','day.4','year.4','month.4','hour.4','wd.4','station.4'], axis=1)
df = df.drop(labels=['No.5','day.5','year.5','month.5','hour.5','wd.5','station.5'], axis=1)
# df = df.drop(labels=['No.6','day.6','year.6','month.6','hour.6','wd.6','station.6'], axis=1)
# df = df.drop(labels=['No.7','day.7','year.7','month.7','hour.7','wd.7','station.7'], axis=1)
# df = df.drop(labels=['No.8','day.8','year.8','month.8','hour.8','wd.8','station.8'], axis=1)
# df = df.drop(labels=['No.9','day.9','year.9','month.9','hour.9','wd.9','station.9'], axis=1)
# df = df.drop(labels=['No.10','day.10','year.10','month.10','hour.10','wd.10','station.10'], axis=1)
# df = df.drop(labels=['No.11','day.11','year.11','month.11','hour.11','wd.11','station.11'], axis=1)
# df.dropna(inplace=True)
data = data_set_util.clean_dataset(df)
data = data_set_util.series_to_supervised_mimo(data, 1, 1)
data.head()


,PM2.5(t-1),PM10(t-1),SO2(t-1),NO2(t-1),CO(t-1),O3(t-1),TEMP(t-1),PRES(t-1),DEWP(t-1),RAIN(t-1),...,PM10.5(t),SO2.5(t),NO2.5(t),CO.5(t),O3.5(t),TEMP.5(t),PRES.5(t),DEWP.5(t),RAIN.5(t),WSPM.5(t)
24,24.0,24.0,26.0,54.0,600.0,36.0,-0.2,1030.5,-17.4,0.0,...,18.0,24.0,43.0,1399.0,25.0,-0.3,1028.9,-18.0,0.0,1.4
25,22.0,24.0,24.0,44.0,500.0,44.0,-0.4,1031.0,-17.6,0.0,...,39.0,25.0,28.0,1300.0,37.0,-0.7,1029.2,-17.9,0.0,1.0
29,14.0,17.0,21.0,36.0,400.0,50.0,-1.0,1031.3,-17.3,0.0,...,23.0,26.0,11.0,1200.0,50.0,-1.3,1028.0,-18.4,0.0,1.4
31,9.0,11.0,19.0,70.0,500.0,20.0,-1.8,1030.1,-17.5,0.0,...,6.0,19.0,11.0,1100.0,52.0,-1.3,1028.0,-17.9,0.0,0.6
32,3.0,11.0,34.0,58.0,500.0,27.0,-1.7,1029.8,-17.0,0.0,...,9.0,22.0,15.0,1200.0,51.0,-0.6,1027.7,-17.8,0.0,0.9


In [5]:
# from sklearn.model_selection import train_test_split
# x,y=data.loc[:,:],data.loc[:,:]
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
# # Get the number of inputs and outputs from the dataset
# df_trainx=x_train.loc[:,'PM2.5(t-1)':'WSPM(t-1)']
# df_testx=x_test.loc[:,'PM2.5(t-1)':'WSPM(t-1)']
# df_trainy=y_train.loc[:,'PM2.5(t)':'WSPM(t)']
# df_original=y_test.loc[:,'PM2.5(t)':'WSPM(t)']

# n_inputs, n_outputs = x.shape[1], y.shape[1]

In [6]:
# MultiOutputRegressor(estimator=RandomForestRegressor(criterion='friedman_mse',
#                                                      max_depth=3,
#                                                      max_leaf_nodes=10,
#                                                      min_samples_leaf=4,
#                                                      min_samples_split=4,
#                                                      n_estimators=50))

def create_and_fit_model(X_train,y_train):
    clf = MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=30,
                                                     max_leaf_nodes=30,
                                                     min_samples_leaf=2,
                                                     min_samples_split=5))
    clf.fit(X_train, y_train)
    
    return clf

In [7]:
def forecast_model(clf, X_test):
    forecast = clf.predict(X_test)
    return forecast

In [8]:
def sliding_window(data,n_windows,train_size):

    result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }
    
    final_result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }

    tam = len(data)
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, train_size, inc=1):
        if len(ttest) > 0:
            
            print('-' * 20)
            print(f'training window {(ct)}')
            
            scaler = StandardScaler()

            
#             Xtrain = ttrain.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
#             ytrain = ttrain.loc[:,'Appliances(t)':'Tdewpoint(t)']
#             Xtest = ttest.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
#             ytest = ttest.loc[:,'Appliances(t)':'Tdewpoint(t)']
            
            Xtrain = scaler.fit_transform(ttrain.loc[:,'PM2.5(t-1)':'WSPM.5(t-1)'])
            ytrain = scaler.fit_transform(ttrain.loc[:,'PM2.5(t)':'WSPM.5(t)'])
            Xtest = scaler.transform(ttest.loc[:,'PM2.5(t-1)':'WSPM.5(t-1)'])
            ytest = scaler.transform(ttest.loc[:,'PM2.5(t)':'WSPM.5(t)'])
            
            
            model = create_and_fit_model(Xtrain,ytrain)
            forecast = forecast_model(model, Xtest)
             
            df_forecats_columns = ttest.loc[:,'PM2.5(t)':'WSPM.5(t)'].columns
            
            columns = list(df_forecats_columns)
            
            prediction = scaler.inverse_transform(forecast) 
            #prediction = forecast
            df_forecast = pd.DataFrame(prediction,columns=columns)
            
            #ytest_metric = scaler.inverse_transform(y_test.reshape(y_test.shape[0], y_test.shape[2]))
            ytest_metric = ttest.loc[:,'PM2.5(t)':'WSPM.5(t)'].values
            df_original = pd.DataFrame(ytest_metric,columns=columns)
            
            for col in columns:  
                original = df_original[col].values
                forecast = df_forecast[col].values
#                 original = original[:len(original)-1]
#                 forecast = forecast[1:]
                
                mae = round(mean_absolute_error(original,forecast),3)
                r2 = round(r2_score(original,forecast),3)
                #rmse = mean_squared_error(original,forecast,squared=False)
                rmse = round(Measures.rmse(original,forecast),3)
                mape = round(Measures.mape(original,forecast),3)
                nrmse = round(cal_nrmse(rmse, original),3)
                smape = round(Measures.smape(original,forecast),3)

                result["rmse"].append(rmse)
                result["nrmse"].append(nrmse)
                result["mape"].append(mape)
                result["mae"].append(mae)
                result["r2"].append(r2)
                result["smape"].append(smape)
                result["window"].append(ct)
                result["variable"].append(col)
                
#                 fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#                 ax.plot(original, label='Original')
#                 ax.plot(forecast, label='Forecast')
#                 handles, labels = ax.get_legend_handles_labels()
#                 lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#                 plt.show()
        
    measures = pd.DataFrame(result)
    return measures

In [9]:
n_windows = 30
train_size = 0.75 

model_rf = sliding_window(data,n_windows,train_size)

--------------------
training window 0


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWar

--------------------
training window 774


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWar

--------------------
training window 1548


...

/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
<ipython-input-3-1be783193a86>:3: RuntimeWarning: divide by zero encountered in double_scalars
  return (rmse/x)
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecast

--------------------
training window 21672


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
<ipython-input-3-1be783193a86>:3: RuntimeWarning: invalid value encountered in double_scalars
  return (rmse/x)
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: invalid value encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
<ip

--------------------
training window 22446


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: invalid value encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: invalid value encountered in true_divide
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: Runtim

In [10]:
df_forecats_columns = data.loc[:,'PM2.5(t)':'WSPM.5(t)'].columns

columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": [],
    "rmse_std": [],
    "mae_std": [],
    "mape_std": [],
    "r2_std": [],
    "smape_std": [],
    "nrmse_std": []
}

model_rf.to_csv (r'all_windows_random_forest_uci_air_quality_beijing_6_sites.csv', index = False, header=True)

measures = model_rf
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    
    rmse_std = round(statistics.stdev(var_agr.loc[:,'rmse']),3)
    mape_std = round(statistics.stdev(var_agr.loc[:,'mape']),3)
    mae_std = round(statistics.stdev(var_agr.loc[:,'mae']),3)
    r2_std = round(statistics.stdev(var_agr.loc[:,'r2']),3)
    smape_std = round(statistics.stdev(var_agr.loc[:,'smape']),3)
    nrmse_std = round(statistics.stdev(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
    
    final_result["rmse_std"].append(rmse_std)
    final_result["mape_std"].append(mape_std)
    final_result["mae_std"].append(mae_std)
    final_result["r2_std"].append(r2_std)
    final_result["smape_std"].append(smape_std)
    final_result["nrmse_std"].append(nrmse_std)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures = round(pd.DataFrame(final_result),3)

In [11]:
final_measures

,variable,rmse,mae,mape,r2,smape,nrmse,rmse_std,mae_std,mape_std,r2_std,smape_std,nrmse_std
0,PM2.5(t),26.219,15.340,32.526,0.838,11.667,0.092,16.558,7.789,15.498,0.108,3.078,0.027
1,PM10(t),42.328,26.075,40.457,0.720,14.089,0.110,26.877,14.890,23.742,0.197,4.359,0.035
2,SO2(t),6.043,3.633,32.883,0.698,12.674,0.104,4.821,2.566,17.083,0.159,4.303,0.023
3,NO2(t),14.711,10.235,24.537,0.762,9.914,0.106,5.658,3.520,11.716,0.104,2.431,0.024
4,CO(t),369.661,234.666,24.989,0.751,9.942,0.103,180.696,112.989,14.959,0.116,2.898,0.024
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,TEMP.5(t),486.749,242.875,inf,-31241.490,22.370,8.798,1556.927,700.675,NaN,160520.064,28.832,37.845
62,PRES.5(t),1.708,1.164,0.115,0.789,0.058,0.090,1.522,1.071,0.104,0.355,0.052,0.062
63,DEWP.5(t),2.323,1.608,inf,0.782,14.164,0.108,2.383,1.732,NaN,0.285,13.104,0.065
64,RAIN.5(t),0.415,0.115,NaN,-65.293,NaN,NaN,0.845,0.231,NaN,358.120,NaN,NaN


In [12]:
pd.set_option('display.max_rows', None)
final_measures

,variable,rmse,mae,mape,r2,smape,nrmse,rmse_std,mae_std,mape_std,r2_std,smape_std,nrmse_std
0,PM2.5(t),26.219,15.340,32.526,0.838,11.667,0.092,16.558,7.789,15.498,0.108,3.078,0.027
1,PM10(t),42.328,26.075,40.457,0.720,14.089,0.110,26.877,14.890,23.742,0.197,4.359,0.035
2,SO2(t),6.043,3.633,32.883,0.698,12.674,0.104,4.821,2.566,17.083,0.159,4.303,0.023
3,NO2(t),14.711,10.235,24.537,0.762,9.914,0.106,5.658,3.520,11.716,0.104,2.431,0.024
4,CO(t),369.661,234.666,24.989,0.751,9.942,0.103,180.696,112.989,14.959,0.116,2.898,0.024
5,O3(t),347.689,59.745,112.520,0.746,20.650,0.120,1803.304,257.405,100.248,0.194,8.192,0.035
6,TEMP(t),495.506,264.772,inf,-22457.219,23.168,8.014,1700.751,790.109,NaN,115767.049,29.287,34.684
7,PRES(t),817.061,166.860,16.399,-257806.193,0.221,23.654,4466.061,907.684,89.205,1412067.068,0.904,129.070
8,DEWP(t),2.413,1.664,inf,0.761,13.801,0.112,2.478,1.810,NaN,0.297,12.640,0.065
9,RAIN(t),0.518,0.176,NaN,-78.691,NaN,NaN,0.832,0.320,NaN,377.929,NaN,NaN


In [13]:
#final_measures.to_csv (r'random_forest_uci_air_quality_beijing.csv', index = False, header=True)

In [15]:
final_measures.to_csv (r'random_forest_uci_air_quality_beijing_6_sites_with_std.csv', index = False, header=True)

In [51]:
#final_measures.to_csv (r'random_forest_uci_air_quality_beijing_1_site.csv', index = False, header=True)

In [53]:
# y_p=clf.predict(df_testx)
# columns=df_original.columns
# y_p=pd.DataFrame(y_p,columns=columns)
# y_p

In [54]:
# y_p.shape,df_original.shape

In [55]:

# col=df_original.columns
# result = {
        
#          "rmse": [],
         
#          "mae": [],
#          "r2": [],
#          "mape":[],
#          "variable":[]
#     }
    
# final_result = {
        
#          "rmse": [],
         
#          "mae": [],
#          "r2": [],
#          "mape":[],
#          "variable":[]}
# for col in col: 
#   original = df_original[col].values
#   forecast = y_p[col].values
#   mae = round(mean_absolute_error(original,forecast),3)
#   r2 = round(r2_score(original,forecast),3)
#   rmse = round(mean_squared_error(original,forecast,squared=False),3)             
#   mape = round(mean_absolute_percentage_error(original,forecast),3)

#   result["rmse"].append(rmse)
#   result["mae"].append(mae)
#   result["r2"].append(r2)
#   result["mape"].append(mape)
  
#   result["variable"].append(col)
        
# measures = pd.DataFrame(result)

In [57]:

# import statistics
# final_result = {
#     "variable": [],
#     "rmse": [],
#     # "nrmse": [],
#     "mae": [],
#     "r2": [],
#     "mape":[]
# }


# var = measures.groupby("variable")

# for col in columns:
    
#     var_agr = var.get_group(col)
           
#     rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
#     # nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
#     mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
#     r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
#     mape = round(statistics.mean(var_agr.loc[:,'mape']),3)

#     final_result["variable"].append(col)
#     final_result["rmse"].append(rmse)
#     # final_result["nrmse"].append(nrmse)
#     final_result["mae"].append(mae)
#     final_result["r2"].append(r2)
#     final_result["mape"].append(mape)
        
#     print(f'Results: {(col,rmse,mae,r2,mape)}')
        
        
# final_measures_kpca = pd.DataFrame(final_result) 

# print("Statistics MIMO (test): ")
# final_measures_kpca

## RandomizedSearchCV

In [9]:
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
# Define the hyperparameter configuration space
hyperparameters = dict(
                     estimator__n_estimators=[20, 50, 100, 200, 300],
                     estimator__criterion=['friedman_mse', 'mse'], estimator__min_samples_split=list(range(2,11)),
                     estimator__max_depth=list(range(2,30)), estimator__min_samples_leaf=list(range(1,11)),
                     estimator__max_leaf_nodes=[5, 10, 20, 30, 50])

clf=RandomForestRegressor()
model=MultiOutputRegressor(clf)
gsc = RandomizedSearchCV(model,hyperparameters,cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1,n_iter=5)

hyperparameters_tuning = gsc.fit(df_trainx, df_trainy)
print('Best Parameters = {}'.format(hyperparameters_tuning.best_params_))

tuned_model = hyperparameters_tuning.best_estimator_

Best Parameters = {'estimator__n_estimators': 50, 'estimator__min_samples_split': 4, 'estimator__min_samples_leaf': 4, 'estimator__max_leaf_nodes': 10, 'estimator__max_depth': 3, 'estimator__criterion': 'friedman_mse'}


In [10]:
hyperparameters_tuning.best_estimator_

MultiOutputRegressor(estimator=RandomForestRegressor(criterion='friedman_mse',
                                                     max_depth=3,
                                                     max_leaf_nodes=10,
                                                     min_samples_leaf=4,
                                                     min_samples_split=4,
                                                     n_estimators=50))

In [11]:
y_p=tuned_model.predict(df_testx)
columns=df_original.columns
y_p=pd.DataFrame(y_p,columns=columns)
y_p

,PM2.5(t),PM10(t),SO2(t),NO2(t),CO(t),O3(t),TEMP(t),PRES(t),DEWP(t),RAIN(t),WSPM(t)
0,92.427193,98.964945,8.837987,79.584022,1368.157050,14.955611,-1588.892874,1027.007135,-7.785169,0.017265,1.343193
1,16.001981,24.597831,8.713903,18.097064,453.649768,1148.239454,18.377070,1027.007135,-14.565495,0.017265,2.831171
2,107.647053,139.241703,45.510944,79.584022,1970.623482,28.172219,7.117797,1027.007135,-14.565495,0.017265,0.907162
3,107.647053,124.787274,23.708185,47.414766,846.605695,104.815486,18.377070,1008.222895,0.764119,0.017265,1.732949
4,264.091984,225.638706,3.226949,128.805978,2261.826394,9.631462,18.377070,1019.009820,13.420079,0.018586,0.838131
...,...,...,...,...,...,...,...,...,...,...,...
4274,86.587682,124.787274,16.462366,47.414766,1332.964637,159.219663,18.377070,1008.133087,15.292069,0.018586,1.514034
4275,42.685794,52.795881,3.226949,47.078241,453.649768,50.010760,18.377070,999.204629,21.519027,0.027173,0.949569
4276,16.518439,86.175741,8.713903,16.574678,453.649768,82.638570,18.377070,1008.133087,-14.565495,0.017265,3.803631
4277,290.340439,309.706078,76.187898,99.358334,3400.302109,91.189205,15.989821,1017.636044,-7.785169,0.017265,0.949569


In [12]:
col=df_original.columns
result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]
    }
    
final_result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]}
for col in col: 
  original = df_original[col].values
  forecast = y_p[col].values
  mae = round(mean_absolute_error(original,forecast),3)
  r2 = round(r2_score(original,forecast),3)
  rmse = round(mean_squared_error(original,forecast,squared=False),3)             
  mape = round(mean_absolute_percentage_error(original,forecast),3)

  result["rmse"].append(rmse)
  result["mae"].append(mae)
  result["r2"].append(r2)
  result["mape"].append(mape)
  
  result["variable"].append(col)
        
measures = pd.DataFrame(result)

In [13]:
import statistics
final_result = {
    "variable": [],
    "rmse": [],
    # "nrmse": [],
    "mae": [],
    "r2": [],
    "mape":[]
}


var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    # nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    # final_result["nrmse"].append(nrmse)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["mape"].append(mape)
        
    print(f'Results: {(col,rmse,mae,r2,mape)}')
        
        
final_measures_kpca = pd.DataFrame(final_result) 

print("Statistics MIMO (test): ")
final_measures_kpca

Results: ('PM2.5(t)', 27.198, 14.039, 0.888, 0.425)
Results: ('PM10(t)', 38.545, 21.717, 0.826, 0.39)
Results: ('SO2(t)', 6.936, 3.655, 0.871, 0.372)
Results: ('NO2(t)', 15.782, 10.386, 0.827, 0.317)
Results: ('CO(t)', 465.153, 247.181, 0.86, 0.32)
Results: ('O3(t)', 159.893, 26.459, -6.828, 1.486)
Results: ('TEMP(t)', 759.897, 87.127, -0.005, 510656520284094.6)
Results: ('PRES(t)', 1965.623, 143.329, -35669.376, 0.141)
Results: ('DEWP(t)', 2.296, 1.666, 0.972, 13070001400243.44)
Results: ('RAIN(t)', 0.542, 0.073, 0.546, 119619676070108.61)
Results: ('WSPM(t)', 0.807, 0.592, 0.547, 151055053163634.38)
Statistics MIMO (test): 


,variable,rmse,mae,r2,mape
0,PM2.5(t),27.198,14.039,0.888,4.250000e-01
1,PM10(t),38.545,21.717,0.826,3.900000e-01
2,SO2(t),6.936,3.655,0.871,3.720000e-01
3,NO2(t),15.782,10.386,0.827,3.170000e-01
4,CO(t),465.153,247.181,0.860,3.200000e-01
5,O3(t),159.893,26.459,-6.828,1.486000e+00
6,TEMP(t),759.897,87.127,-0.005,5.106565e+14
7,PRES(t),1965.623,143.329,-35669.376,1.410000e-01
8,DEWP(t),2.296,1.666,0.972,1.307000e+13
9,RAIN(t),0.542,0.073,0.546,1.196197e+14


## MinMaxScaler

In [14]:
x,y=data.loc[:,:],data.loc[:,:]

In [15]:
from sklearn.preprocessing import MinMaxScaler
scalerx = MinMaxScaler()
scalery=MinMaxScaler()

In [16]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
# Get the number of inputs and outputs from the dataset
df_trainx=x_train.loc[:,'PM2.5(t-1)':'WSPM(t-1)']
df_trainx_scaled = scalerx.fit_transform(df_trainx)
df_testx=x_test.loc[:,'PM2.5(t-1)':'WSPM(t-1)']
df_testx_scaled = scalerx.transform(df_testx)
df_trainy=y_train.loc[:,'PM2.5(t)':'WSPM(t)']
df_trainy_scaled=scalery.fit_transform(df_trainy)

df_original=y_test.loc[:,'PM2.5(t)':'WSPM(t)']

n_inputs, n_outputs = df_trainx.shape[1], df_trainy.shape[1]

In [17]:
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
# Define the hyperparameter configuration space
hyperparameters = dict(
                     estimator__n_estimators=[20, 50, 100, 200, 300],
                     estimator__min_samples_split=list(range(2,11)),
                     estimator__max_depth=[2,5,10,15,20,30], estimator__min_samples_leaf=list(range(1,11)),
                     estimator__max_leaf_nodes=[5, 10, 20, 30, 50])

clf=RandomForestRegressor()
model=MultiOutputRegressor(clf)
gsc = RandomizedSearchCV(model,hyperparameters,cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1,n_iter=5)

hyperparameters_tuning = gsc.fit(df_trainx_scaled, df_trainy_scaled)
print('Best Parameters = {}'.format(hyperparameters_tuning.best_params_))

tuned_model = hyperparameters_tuning.best_estimator_

Best Parameters = {'estimator__n_estimators': 100, 'estimator__min_samples_split': 5, 'estimator__min_samples_leaf': 2, 'estimator__max_leaf_nodes': 30, 'estimator__max_depth': 30}


In [18]:
hyperparameters_tuning.best_estimator_

MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=30,
                                                     max_leaf_nodes=30,
                                                     min_samples_leaf=2,
                                                     min_samples_split=5))

In [19]:
y_p=tuned_model.predict(df_testx_scaled)
columns=df_original.columns
y_p=pd.DataFrame(y_p,columns=columns)
y_p

,PM2.5(t),PM10(t),SO2(t),NO2(t),CO(t),O3(t),TEMP(t),PRES(t),DEWP(t),RAIN(t),WSPM(t)
0,0.122443,0.103021,0.037069,0.266563,0.141746,0.000210,0.251689,0.000035,0.419354,0.000220,0.155084
1,0.012437,0.015227,0.034580,0.064752,0.032086,0.080667,0.294255,0.000032,0.330061,0.000233,0.365713
2,0.150735,0.146042,0.240174,0.267232,0.182637,0.000338,0.291560,0.000035,0.330705,0.000220,0.110498
3,0.150502,0.132841,0.115045,0.150532,0.083518,0.001451,0.294255,0.000020,0.585383,0.000220,0.200028
4,0.340499,0.250084,0.008507,0.464516,0.224501,0.000058,0.295128,0.000027,0.757911,0.000427,0.101120
...,...,...,...,...,...,...,...,...,...,...,...
4274,0.121133,0.128729,0.094919,0.152885,0.111547,0.002217,0.294255,0.000014,0.769633,0.000932,0.175719
4275,0.063300,0.048850,0.008507,0.148905,0.045881,0.000741,0.294730,0.000014,0.863209,0.001524,0.118681
4276,0.029690,0.095327,0.034580,0.031984,0.032086,0.000810,0.294255,0.000021,0.335635,0.000220,0.473029
4277,0.446140,0.339041,0.436649,0.331229,0.324645,0.000993,0.291359,0.000031,0.413866,0.000220,0.113551


In [20]:
y_p_inv = scalery.inverse_transform(y_p)
columns=df_original.columns
y_p_inv=pd.DataFrame(y_p_inv,columns=columns)
y_p_inv

,PM2.5(t),PM10(t),SO2(t),NO2(t),CO(t),O3(t),TEMP(t),PRES(t),DEWP(t),RAIN(t),WSPM(t)
0,89.934537,100.354624,7.820752,78.770009,1503.282469,17.421429,-1273.640366,1023.573017,-8.545235,0.006432,1.333721
1,11.830117,17.389322,7.362725,20.648616,417.656142,6307.699637,20.363931,1019.673704,-14.242123,0.006816,3.145134
2,110.021844,141.010070,45.191998,78.962755,1908.108509,27.434991,-61.583978,1023.130476,-14.200990,0.006432,0.950279
3,109.856330,128.535111,22.168229,45.353111,926.832743,114.427012,20.363931,1008.303842,2.047433,0.006432,1.720241
4,244.754343,239.329783,2.565204,135.780558,2322.558148,5.561859,46.879087,1015.029582,13.054710,0.012510,0.869631
...,...,...,...,...,...,...,...,...,...,...,...
4274,89.004336,124.649251,18.465151,46.030934,1204.312962,174.304567,20.363931,1002.151877,13.802586,0.027297,1.511182
4275,47.943290,49.163270,2.565204,44.884570,554.219088,58.904335,34.783976,1001.922516,19.772721,0.044643,1.020658
4276,24.079761,93.083753,7.362725,11.211281,417.656142,64.358693,20.363931,1009.054847,-13.886493,0.006432,4.068047
4277,319.759124,323.393977,81.343387,97.393856,3313.988339,78.626635,-67.700538,1019.648274,-8.895378,0.006432,0.976540


In [21]:
df_original

,PM2.5(t),PM10(t),SO2(t),NO2(t),CO(t),O3(t),TEMP(t),PRES(t),DEWP(t),RAIN(t),WSPM(t)
34010,94.0,96.0,16.0,71.0,1600.0,17.0,-3275.00,1028.00,-8.1,0.0,1.5
33853,7.0,7.0,4.0,20.0,400.0,65.0,2.85,1022.75,-14.3,0.0,2.2
16507,115.0,142.0,37.0,93.0,1900.0,7.0,-2.00,1025.00,-13.5,0.0,0.7
9783,95.0,135.0,21.0,49.0,800.0,128.0,20.80,1007.80,2.1,0.0,2.9
31752,225.0,225.0,2.0,145.0,2400.0,2.0,14.00,1015.30,13.2,0.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...
22411,72.0,94.0,20.0,45.0,1100.0,161.0,24.80,1006.50,13.7,0.0,1.9
11389,28.0,28.0,2.0,22.0,400.0,125.0,30.60,995.70,18.0,0.0,1.1
7336,16.0,94.0,4.0,7.0,400.0,64.0,8.30,1006.60,-14.0,0.0,2.9
34463,317.0,317.0,73.0,99.0,3200.0,73.0,-0.10,1016.50,-8.3,0.0,1.1


In [22]:
col=df_original.columns
result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]
    }
    
final_result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]}
for col in col: 
  original = df_original[col].values
  forecast = y_p_inv[col].values
  mae = round(mean_absolute_error(original,forecast),3)
  r2 = round(r2_score(original,forecast),3)
  rmse = round(mean_squared_error(original,forecast,squared=False),3)             
  mape = round(mean_absolute_percentage_error(original,forecast),3)

  result["rmse"].append(rmse)
  result["mae"].append(mae)
  result["r2"].append(r2)
  result["mape"].append(mape)
  
  result["variable"].append(col)
        
measures = pd.DataFrame(result)

In [23]:
import statistics
final_result = {
    "variable": [],
    "rmse": [],
    # "nrmse": [],
    "mae": [],
    "r2": [],
    "mape":[]
}


var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    # nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    # final_result["nrmse"].append(nrmse)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["mape"].append(mape)
        
    print(f'Results: {(col,rmse,mae,r2,mape)}')
        
        
final_measures_kpca = pd.DataFrame(final_result) 

print("Statistics MIMO (test): ")
final_measures_kpca

Results: ('PM2.5(t)', 26.125, 12.651, 0.897, 0.347)
Results: ('PM10(t)', 37.738, 20.107, 0.833, 0.342)
Results: ('SO2(t)', 6.742, 3.305, 0.878, 0.299)
Results: ('NO2(t)', 14.942, 9.513, 0.845, 0.287)
Results: ('CO(t)', 423.771, 217.984, 0.884, 0.264)
Results: ('O3(t)', 620.883, 41.189, -117.042, 1.423)
Results: ('TEMP(t)', 768.415, 102.348, -0.027, 700775132244580.9)
Results: ('PRES(t)', 5310.394, 181.862, -260350.842, 0.18)
Results: ('DEWP(t)', 1.674, 0.95, 0.985, 5123938688391.475)
Results: ('RAIN(t)', 0.544, 0.072, 0.543, 117531142881785.92)
Results: ('WSPM(t)', 0.795, 0.58, 0.56, 139664280467138.14)
Statistics MIMO (test): 


,variable,rmse,mae,r2,mape
0,PM2.5(t),26.125,12.651,0.897,3.470000e-01
1,PM10(t),37.738,20.107,0.833,3.420000e-01
2,SO2(t),6.742,3.305,0.878,2.990000e-01
3,NO2(t),14.942,9.513,0.845,2.870000e-01
4,CO(t),423.771,217.984,0.884,2.640000e-01
5,O3(t),620.883,41.189,-117.042,1.423000e+00
6,TEMP(t),768.415,102.348,-0.027,7.007751e+14
7,PRES(t),5310.394,181.862,-260350.842,1.800000e-01
8,DEWP(t),1.674,0.950,0.985,5.123939e+12
9,RAIN(t),0.544,0.072,0.543,1.175311e+14
